# Transformée en Ondelettes 2D, application au traitement des images

##    NGUYEN Minh Hai - LE Cam Thanh Ha - 4 GMM A

In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import matplotlib.pyplot as plt
import pywt
import scipy.io as sio
import pandas as pd
import holoviews as hv
import param
import panel as pn
from panel.pane import LaTeX
hv.extension('bokeh')
from PIL import Image
from io import BytesIO
import requests

## Approximation linéaire et non linéaire.

In [ ]:
local=0
def chargeData(name):
    if local:
        if name=='Lenna':
            res=np.array(Image.open("./Archive/img/Lenna.jpg")).astype(float)
        if name=='Canaletto':
            res=np.array(Image.open("./Archive/img/Canaletto.jpeg")).astype(float)
        if name=='Minotaure':
            res=np.array(Image.open("./Archive/img/MinotaureBruite.jpeg")).astype(float)   
        if name=='Cartoon':
            res=np.array(Image.open("./Archive/img/Cartoon.jpg")).astype(float) 
    else:
        if name=='Lenna':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Lenna.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Canaletto':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Canaletto.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Minotaure':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/MinotaureBruite.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Cartoon':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Cartoon.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
    return res

In [ ]:
lena = chargeData('Lenna')
cana = chargeData('Canaletto')
imagesRef = {"Lenna" : lena, "Canaletto" : cana}
options = dict(cmap = 'gray', xaxis=None, yaxis=None, width=400, height=400, toolbar=None)
pn.Row(hv.Raster(imagesRef["Lenna"]).opts(**options), hv.Raster(imagesRef["Canaletto"]).opts(**options)) 

### Transformation en ondelettes des images

In [ ]:
size = 400
WT = pywt.wavedecn(cana, 'haar', mode='per', level=2)
arr, coeff_slices = pywt.coeffs_to_array(WT)
hv.Image(arr).opts(cmap='gray',width=size,height=size)

<span style='color:blue'>
        
On peut constater que la transformation en ondelettes de cette image contient beaucoup de coefficients nuls dans les espaces de détails $W_j$. Donc on peut faire une approximation dans la famille multi-résolution $V_j$
    
<span> 

### Approximation Linéaire 

Ecrire une fonction qui réalise une approxiamtion non linéaire en seuillant les coefficients d'ondelettes.
On pourra utiliser les fonctions suivante : pywt.coeffs_to_array et pywt.array_to_coeffs

In [ ]:
def ApproxOnd2DThres(S, qmf, L, threshold):
    '''
    Non-Linear approximation in Wavelet base, using threshold
    S (arr): input signal
    qmf (str) : wavelet name
    L (int) : maximum level
    '''
    Wavelet = pywt.wavedecn(data = S, wavelet = qmf, mode = 'per', level = L)
    arr, coef_slices = pywt.coeffs_to_array(Wavelet)
    
    ncoefs = np.sum(np.abs(arr) > threshold)  # Number of coefs that we retained
    Wavelet_thres = arr*(np.abs(arr) > threshold)
    
    coef_from_arr = pywt.array_to_coeffs(Wavelet_thres, coef_slices)
    Srec = pywt.waverecn(coef_from_arr, qmf, mode = 'per')
    
    return Srec, ncoefs


<span style='color:blue'>
        
Focntion PSNR pour mesurer la qualité des images bruitées par rapport à l'image initial
    
<span> 



In [ ]:
def PSNR(I, Iref):
    '''
    Peak signal-to-noise ration for 8bit images
    I - reconstructed image
    Iref - original image
    '''
    MSE = np.mean((I - Iref)**2)
    if MSE == 0: # There is no noise
        return 100
    psnr = 20*np.log10(255) - 10*np.log10(MSE)
    return psnr
    

Ecrire une fonction qui réalise une approximation non linéaire en conservant un nombre N de coefficients d'ondelettes et la tester. On pourra utiliser les fonctions pywt.ravel_coeffs et unravel_coeffs.

In [ ]:
def ApproxOnd2Num(S,qmf,L,N):
    '''
    Non-linear approximation in Wavelet base, using number of coefs
    '''
    Wavelet = pywt.wavedecn(data = S, wavelet = qmf, mode = 'per', level = L)
    arr, coef_slices = pywt.coeffs_to_array(Wavelet)
    arr_flat = arr.flatten()
    if N > np.product(arr.shape):
        N = np.product(arr.shape)
    index = np.argsort(np.abs(arr_flat))[-N:]
    arr_flat_approx = np.zeros(arr_flat.shape)
    arr_flat_approx[index] = arr_flat[index]
    arr_approx = arr_flat_approx.reshape(arr.shape)
    
    coef_from_arr = pywt.array_to_coeffs(arr_approx, coef_slices)
    Srec = pywt.waverecn(coef_from_arr, qmf, mode = 'per')
    return Srec

### Approximation Linéaire en bases ondelettes

In [ ]:
N = 10000
Cana_rec = ApproxOnd2Num(cana,'db2',6,N)
pn.Row(hv.Image(cana).opts(cmap='gray',width=400,height=400, title = 'Original Image'), hv.Image(Cana_rec).opts(cmap='gray',width=400,height=400, title = 'Non-linear approximation with ' + str(N) + ' coefs', ))

<span style='color:blue'>
    
En gardant les 10000 plus grands coefficients dans l'espace d'approximation $V_j$, on obtient une approximation assez proche de l'image original. 
    
    Nous allons regarder l'évolution de PSNR en fonction de nombre de coefficients qu'on garde. 
    
<span>    

In [ ]:
psnr_cana = []
for n in range(1, 512*512, 2000):
    psnr_cana.append(PSNR(cana,ApproxOnd2Num(cana,'db2',6,n)))

In [ ]:
hv.Curve((range(1,512*512,2000), psnr_cana)).opts(width = 800, title = "PSNR of non-linear approx. for Canaletto", xlabel = 'Number of coef', ylabel = 'PSNR')

<span style='color:blue'>
    
Il est clair que plus de coefficient on garde, meilleure qualité de l'approximation on obtient    
    
<span>

In [ ]:
T = 5
Cana_rec_thres, ncoefs = ApproxOnd2DThres(cana,'db2',6, T)
pn.Row(hv.Image(cana).opts(cmap='gray',width=500,height=500, title = 'Original Image'), hv.Image(Cana_rec_thres).opts(cmap='gray',width=500,height=500, title = 'Linear approximation with threshold egal to ' + str(T)))

In [ ]:
psnr_cana_thres = []
for t in range(1,200):
    psnr_cana_thres.append(PSNR(cana,ApproxOnd2DThres(cana,'db2',6, t)[0]))

In [ ]:
hv.Curve((range(1,200), psnr_cana_thres)).opts(width = 800, title = "PSNR of non-linear approx.", xlabel = 'Threshold', ylabel = 'PSNR')

<span style='color:blue'>
    
Similairement avec l'approximation utilisant un seuil. Plus le seuilt est petite, plus les coefficients on garde donc meilleur qualité d'approximation
    
<span>

### Dashboard pour visualiser

Créer un Dashboard qui permet d'explorer la fonction précédente.

In [ ]:
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']
imagesRef = {"Lenna" : lena, "Canaletto" : cana}

class Approx2D(param.Parameterized):
    image = param.ObjectSelector(default = "Canaletto", objects = imagesRef.keys())
    wave = param.ObjectSelector(default = "haar", objects = wavelist)
    L = param.Integer(5,bounds=(0,7))
    N = param.Integer(2000,bounds=(1,512*512))
    #  @param.depends('wave', 'N', 'L')
    
    def view(self):
        S = imagesRef[self.image]
        Srec = ApproxOnd2Num(S, self.wave, self.L, self.N)
        return pn.Row(hv.Image(S).opts(cmap='gray',width=400,height=400, title = 'Original Image'), hv.Image(Srec).opts(cmap='gray',width=400,height=400, title = 'Non-linear approximation with ' + str(self.N) + ' coefs', ))
    
    
       

In [ ]:
approx2D = Approx2D()
pn.Row(approx2D.param, approx2D.view)

<span style='color:blue'>

Quelques remarques :     
-  Avec le même nombre de coefficients, plus de transformation en ondelettes on fait, plus d'information on perd. En effet, on considède l'image original comme un 
    élément de l'espace $V_{j}$ et donc si on fait plus de transformation en ondelettes, on est on train de projeter l'image sur un espace $V_{j'}$ de plus petite dimension, 
    donc l'approximation est moins bonne car on perd plus d'informations sur les espaces de détails $W_j$
-  Il y a des effets 'carrés' pour la base de Haar, ce n'est pas étonnant car la base de Haar sont des fonctions constantes par morceaux.
-  Les ondelettes de Daubechies 2 semblent plus pertinentes
    
<span>

### Plan d'expériences

Créer un plan d'experiences qui permet d'explorer la fonction ApproxOnd2DNum

In [ ]:
import itertools
wavelist = ['haar','db2','db3','db4','coif1','coif2','coif3']
experiences = {'Image': imagesRef, 'num_coefs':np.linspace(1000, 50000, 40, dtype = 'int'), 'level': range(4,7), 'wave': wavelist}

dfexp = pd.DataFrame(list(itertools.product(*experiences.values())),columns=experiences.keys())


In [ ]:
print(dfexp)

Créer la fonction qui à une ligne de la base de donnée précédente calcule le PSNR associé.

In [ ]:
def row2PSNR(row):
    S = imagesRef[row.Image]
    Srec = ApproxOnd2Num(S, row.wave, row.level, row.num_coefs)
    return PSNR(S, Srec)

Appliquer la fonction sur la base de donnée et ajouter la colonne PSNR à la base de données dfexp 

In [ ]:
row = dfexp.iloc[1]
row

In [ ]:
row2PSNR(dfexp.iloc[1])

In [ ]:
PSNR_plan = dfexp.apply(row2PSNR, axis = 1)


In [ ]:
PSNR_plan

In [ ]:
dfexp['PSNR'] = PSNR_plan

In [ ]:
print(dfexp)

### Utiliser hvplot pour visualiser la base de données.

In [ ]:
import hvplot.pandas

In [ ]:
from bokeh.models import HoverTool
h = HoverTool()
dfexp.hvplot('num_coefs','PSNR',by='wave',kind='scatter',groupby=['Image', 'level']).opts(width=600,tools = [h]).redim.range(PSNR=(5,100), N = (0,50000))

<span style='color:blue'>
    
En terme de PSNR, les ondelettes de Coifman 3 semblent les plus perfomantes et celles de Haar sont moins efficaces.      

<span>

## Débruitage d'images

Ecrire une fonction qui effectue un seuillage dur en ondelettes et la tester. On pourra utiliser la fonction pywt.ravel_coeffs et on pensera à cliper le résultat entre 0 et 255.

In [ ]:
def SeuillageDurOndelettes(I,qmf,L,Seuil):
    '''
    Like a non-linear approximation
    '''
    Wavelet = pywt.wavedecn(data = I, wavelet = qmf, mode = 'per', level = L)
    arr, coef_slices = pywt.coeffs_to_array(Wavelet)
    
    Wavelet_seuil = arr * (np.abs(arr) > Seuil)
    coef_from_arr = pywt.array_to_coeffs(Wavelet_seuil, coef_slices)
    
    Irec = pywt.waverecn(coef_from_arr, qmf, mode = 'per')
    Irec = np.clip(Irec, 0, 255)
    return Irec

### Dashboard

Construire un dashboard qui permet d'explorer la fonction SeuillageDurOndelettes.

In [ ]:
class WaveSeuillage(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto", objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar", objects=wavelist)
    L = param.Integer(7, bounds=(0,7))
    Seuil = param.Number(10, bounds=(1,1000))
    def view(self):
        S = imagesRef[self.image]
        Srec = SeuillageDurOndelettes(S, self.wave, self.L, self.Seuil)
        print('PSNR of reconstructed image', PSNR(Srec, S))
        return pn.Row(hv.Image(S).opts(cmap='gray',width=400,height=400, title = 'Original Image'), hv.Image(Srec).opts(cmap='gray',width=400,height=400, title = 'Seuillage dur en ondelettes avec un seuil de' + str(self.Seuil)))

In [ ]:
WaveSeuillage = WaveSeuillage()
pn.Row(WaveSeuillage.param, WaveSeuillage.view)

### Ajouter du bruit à l'image de Canaletto

In [ ]:
n1, n2 = np.shape(cana)
B = np.random.randn(n1,n2)
sigma = 10
ib = cana + sigma*B
ib = np.clip(ib,0,255)
hv.Image(ib).opts(cmap='gray', width=400, height=400)


In [ ]:
def addNoise(I, sigma):
    n1, n2 = I.shape
    B = np.random.randn(n1, n2)
    Ibruitee = I + sigma*B
    Ibruitee = np.clip(Ibruitee,0,255)
    return Ibruitee

Ecrire un dashboard qui permet de visualiser rapidement l'effet d'un débruitage en ondelettes et qui renvoie les images originales, bruitées et débruitées ainsi que les PSNR associés aux images bruitéeset débruitées.

In [ ]:
class WaveDebruit(param.Parameterized):
    image = param.ObjectSelector(default="Lenna",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(7,bounds=(0,7))
    Seuil = param.Number(3,bounds=(1,20))
    Sigma = param.Number(10,bounds=(1,30))
    seednoise = param.Integer(1,bounds=(0,50))
    def view(self):
        I = imagesRef[self.image]
        np.random.seed(self.seednoise)
        Ibruitee = addNoise(I, self.Sigma)
        Idebruitee = SeuillageDurOndelettes(Ibruitee, self.wave, self.L, self.Seuil)
        PSNRbruitee = PSNR(Ibruitee, I)
        PSNRdebruitee = PSNR(Idebruitee, I)
        return pn.GridBox(hv.Image(I).opts(cmap='gray',width=400,height=400, title = 'Original Image', fontsize={'title': 10} ), 
                      hv.Image(Ibruitee).opts(cmap='gray',width=400,height=400, title = 'Image bruitée avec sigma = ' + str(self.Sigma) + ', PSNR = ' + str(round(PSNRbruitee,2)), fontsize={'title': 10}),
                      hv.Image(Idebruitee).opts(cmap='gray',width=400,height=400, title = 'Image debruitée avec seuil = ' + str(round(self.Seuil,2)) + ', PSNR = ' + str(round(PSNRdebruitee,2)), fontsize={'title': 10}), ncols = 2)

In [ ]:
waveDebruit = WaveDebruit()
pn.Row(waveDebruit.param, waveDebruit.view)

<span style='color:blue'>    

Commentaires:
-  Avec ...
    
<span>

## Débruitage d'images et translations

Ecrire une fonction qui réalise un débruitage avec une moyenne sur des NbT fois NbT translations et la tester. Vérifier le gain en PNSR.

In [ ]:
def DebruitTrans(S, qmf, seednoise, sigma, T, trans):
    N1 = len(S)
    np.random.seed(seed=seednoise)
    bruit = np.random.normal(0,1,N1)
    Lmax = pywt.dwt_max_level(len(S),pywt.Wavelet(qmf).dec_len)
    SB = S + sigma * bruit
    Seuil = T * sigma
    SSum = 0 * SB
    P = np.zeros(trans)
    for k in np.arange(0,trans):
        SBtemp = np.roll(SB, k)
        Srectemp = SeuillageOndelette(SBtemp, qmf, Lmax, Seuil)
        Srectemp2 = np.roll(Srectemp,-k)
        SSum = SSum + Srectemp2
        Srec = SSum /(k + 1)
        P[k] =psnr(S, Srec)
    return Srec, P

In [ ]:
def DebruitTranslation(IB, wave, L, seuil, NbT):
    Ideb = SeuillageDurOndelettes(IB, wave, L, seuil)
    sumIdeb = 0*IB
    for k in range(NbT):
        IB_k_v = np.roll(IB, k, axis = 0)
        IDB_k_v = SeuillageDurOndelettes(IB_k_v, wave, L, seuil)
        IDB_k_v = np.roll(IDB_k_v, -k, axis = 0)
        
        IB_k_v2 = np.roll(IB, -k, axis = 0)
        IDB_k_v2 = SeuillageDurOndelettes(IB_k_v2, wave, L, seuil)
        IDB_k_v2 = np.roll(IDB_k_v2, k, axis = 0)
        
        IB_k_h = np.roll(IB, k, axis = 1)
        IDB_k_h = SeuillageDurOndelettes(IB_k_h, wave, L, seuil)
        IDB_k_h = np.roll(IDB_k_h, -k, axis = 1)
        
        IB_k_h2 = np.roll(IB, -k, axis = 1)
        IDB_k_h2 = SeuillageDurOndelettes(IB_k_h2, wave, L, seuil)
        IDB_k_h2 = np.roll(IDB_k_h2, k, axis = 1)
        
        sumIdeb = sumIdeb + IDB_k_v + IDB_k_h + IDB_k_v2 + IDB_k_h2

    avgIdeb = sumIdeb / (NbT*4)
    return avgIdeb

Créer un dasboard pour explorer la fonction précédente. La sortie doit aussi être composée de 3 images et 2 PSNR.

In [ ]:
class Debruit_translat(param.Parameterized):
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    L = param.Integer(7,bounds=(0,7))
    Seuil = param.Number(3,bounds=(1,150))
    NbT = param.Integer(2,bounds=(1,8))
    Sigma = param.Number(10,bounds=(1,30))
    seednoise = param.Integer(1,bounds=(0,50))
    
    def view(self):
        I = imagesRef[self.image]
        np.random.seed(self.seednoise)
        Ibruitee = addNoise(I, self.Sigma)
        Idebruitee = SeuillageDurOndelettes(Ibruitee, self.wave, self.L, self.Seuil)
        IDebTrans = DebruitTranslation(Ibruitee, self.wave, self.L, self.Seuil, self.NbT)
        
        PSNRbruitee = PSNR(Ibruitee, I)
        PSNRdebruitee = PSNR(Idebruitee, I)
        PSNR_trans = PSNR(IDebTrans, I)
        return pn.GridBox(hv.Image(I).opts(cmap='gray',width=400,height=400, title = 'Original Image', fontsize={'title': 10} ), 
                      hv.Image(Ibruitee).opts(cmap='gray',width=400,height=400, title = 'Image bruitée avec sigma = %.2f' % self.Sigma + ', PSNR = %.2f' % PSNRbruitee, fontsize={'title': 10}),
                      hv.Image(Idebruitee).opts(cmap='gray',width=400,height=400, title = 'Image debruitée avec seuil = %.2f' % self.Seuil + ', PSNR = %.2f' % PSNRdebruitee, fontsize={'title': 10}), 
                      hv.Image(IDebTrans).opts(cmap='gray',width=400,height=400, title = 'Debruitée trans. avec NbT = %.2f' % self.NbT + ', PSNR = %.2f' % PSNR_trans, fontsize={'title': 10}), ncols = 2)

In [ ]:
debruitTrans = Debruit_translat()
pn.Row(debruitTrans.param, debruitTrans.view)

<span style='color:blue'>

Commentaires:    
-  Cette méthdoe de translation semble plus efficace en terme de PSNR
-  Il faut chercher la compromise entre la perte des détails et la réduction du bruit, si on met un seuil plus haut on réduit plus de bruit mais on perd plus de détails et inversement.
-  Il y a toujours les effets 'carrés' des ondelettes de Haar ce qui sont prévus
-  Pour toutes les bases ondelettes expérimentées, les bords des objets dans l'image sont les parties les plus difficiles pour nettoyer, et il y a encore des pertes d'informations sur les bords
    
<span>

## Débruitage d'une image couleur.

Pour effectuer le débruitage d'une image générale, c'est à dire d'une image couleur dont le format n'est pas carré et dont les dimensions ne sont pas des puissnaces de 2 on procède comme suit :

1) On effectue un débruitage séparé sur chacun des canaux.

2) Le format carré n'est pas un vraiu problème, il faut juste que les dimensions soit des multiples de puissances de 
2. C'est la puissance de 2 qui définira l'échalle maximale de la décomposition en ondelettes. Il est donc préférable que les dimensions de l'images soient un petit multiple d'une puissance de 2.

3) On étend l'image par symétrie ou périodicité pour qu'elle ait les dimensions souhaitées. A la fin du processus de débruitage on tronque le résultat obtenu à la dimension de l'image originale.

4) Si le niveau de bruit n'est pas connu, il faut l'estimer en utilisant les coefficients d'ondelettes de la plus petite échelle (voir le notebook sur le débruitage de signaux).


In [ ]:
def DebuitCouleur(IB, wave = 'coif3', L = 5, seuil = 10, NbT = 4):
    '''
    IB : color noised-image RGB (3 channels), of shape (H, W, C = 3)
    Using translation method for each channel
    '''
    ID = np.zeros(IB.shape)
    for c in range(IB.shape[-1]):
        ID[:,:,c] = DebruitTranslation(IB[:,:,c], wave, L, seuil, NbT)
    
    return ID
    

### Dashboard

In [ ]:
class ColorImageDebruit(param.Parameterized):
    image = param.ObjectSelector(default='MinotaureBruite.jpeg')
    wave = param.ObjectSelector(default="coif3", objects=wavelist)
    L = param.Integer(7, bounds=(0,7))
    Seuil = param.Number(100, bounds=(0,200))
    NbT = param.Integer(2, bounds=(1,8))
    
    def view(self):
        im = np.copy(np.array(Image.open(self.image)))
        h, w, _ = im.shape
        im_deb = DebuitCouleur(im, self.wave, self.L, self.Seuil, self.NbT)
        
        return pn.Row(hv.RGB(im.astype('uint8')).opts(width = w, height = h, title = 'Image bruitée', fontsize={'title': 10} ), 
                      hv.RGB(im_deb.astype('uint8')).opts(width = w, height = h, title = 'Image debruitée', fontsize={'title': 10}))

In [ ]:
Minotaure = chargeData("Minotaure")

In [ ]:
rec_image = ColorImageDebruit(image = 'img/MinotaureBruite.jpeg')
pn.Row(rec_image.param, rec_image.view)

# Proposer une fonction qui effectue le débruitage d'une image couleur de dimensions quelconques. 

La fonction peut prendre en entrée un tableau numpy ou une image dans une format d'images classique.
Vous pouvez tester votre programme en bruitant vous même une ou plusieurs images de référence et évaluer le gain en terme de PSNR.

In [ ]:
class ColorImageDebruitMultidimension(param.Parameterized):
    image = param.ObjectSelector(default = 'MinotaureBruite.jpeg')
    wave = param.ObjectSelector(default = "coif3",objects=wavelist)
    L = param.Integer(7,bounds=(0,7))
    Seuil = param.Number(100,bounds=(0,200))
    Sigma = param.Number(20,bounds=(1,50.1))
    seednoise = param.Integer(1,bounds=(0,50))
    NbT = param.Integer(2,bounds=(1,8))
    
    def view(self):
        im = np.array(Image.open(self.image))
        dim = np.shape(im)[2]
        recim = np.copy(im)

        for i in range(dim):
            canal_temp = im[:,:,i]
            rec_cal_temp = DebruitTranslation(canal_temp, self.wave, self.L, self.Seuil, self.NbT)
            recim[:,:,i] = rec_cal_temp 
        return pn.Row(hv.RGB(recim).opts(**options1) + hv.RGB(im).opts(**options1))

## Pour aller plus loin (à titre informatif et optionnel)

On peut améliorer les méthodes par seuillage dans une base d'ondelettes en effectuant un seuillage par blocs. C'est à dire, ne pas décider de conserver ou pas un coefficients en fonction de sa seule amplitude mais plutôt en fonction de l'énergie d'un voisinage de coefficients. 

Voir : http://www.cnrs.fr/insmi/spip.php?article265

En effet, il est rare qu'un coefficient soit significatif seul au milieu de coefficients nuls. 

La mméthode de sueillage par blocs consiste à choisir une taille de voisinage (par exemple 4*4 coeffients en dimension 2) pour une échelle et une direction donnée et de conserver l'intégralité des coefficients si l'énergie (la somme des carrés des coefficients) est supérieure à un seuil et de les mettre tous à 0 si ce n'est pas le cas. 

Dans ce cas aussi, les translations permettent d'améliorer le rendu visuel en limitant les effets de blocs.

On peut aussi constuire des blocs "3D" en considérant des blocs qui comprennent les coefficients des 3 créneaux de couleurs. L'idée est de corréler le débruitage un peu à travers l'espace et l'espace des couleurs.

Il est possible d'effectuer un débruitage en changeant d'espace colorimétrique en passant du RGB au YUV par exemple.

# Débruiter un minotaure ?

A l'aide de tout ce qui a été fait précédemment, proposer une version débruitée de l'image couleur contenue dans le tableau Mi

In [ ]:
Mi = chargeData('Minotaure')
Minotaure = np.clip(Mi, 0 ,255)
Minotaure_deb = DebuitCouleur(Minotaure, wave = 'db4', L = 5, seuil = 75, NbT = 5)

pn.Row(hv.RGB(Minotaure.astype('uint8')).opts(title = 'Minautaure bruité', width = 400, height = 500),
    hv.RGB(Minotaure_deb.astype('uint8')).opts(title = 'Minautaure debruité', width = 400, height = 500))


<span style = 'color:blue'>
    
Nous avons bien débruité le Minotaure en utilisant les bases ondelettes de Daubechies 4 avec Translation et un seuillage de $75$. L'image débruitée a été réduite significativement sans pertes des détails.
    
<span>

Rédiger également une fonction prenant en entrée un nom de fichier 
permettant de calculer le PSNR de votre proposition d'image débruitée avec l'image en question.
On calcule le PSNR entre deux images couleurs en calculant la somme des erreurs quadratiques sur les 3 canaux.

Attention, l'image a 3 canaux de couleur, n'est pas carrée et les dimensions ne sont pas des puissances de 2.

### Créer un plan d'expériences pour explorer les performances de l'invariance par translation pour le débruitage. 

In [ ]:
experiences_DebruitTrans = {'Image':imagesRef.keys(), 'Seuil':np.linspace(80,120,10), 'NbT':np.arange(3,5),'wave':wavelist,'Sigma':np.linspace(5,10,2)}
dfexp_DebruitTrans = pd.DataFrame(list(itertools.product(*experiences_DebruitTrans.values())), columns = experiences_DebruitTrans.keys())

In [ ]:
print(dfexp_DebruitTrans)

Ecrire une fonction qui calcule le PSNR moyen sur n réalisations de bruit du débruitage d'une image avec NbT*NbT translations (qui utilise par exemple la fonction DebruitTranslation)

In [ ]:
def Debruit_Translat_PSNRMoyen(I, wave, sigma, seuil, NbT, n):
    P = 0
    for seednoise in np.arange(n):
        np.random.seed(seednoise)
        IB = addNoise(I, sigma)
        Irec = DebruitTranslation(IB, wave, 4, seuil, NbT)
        P = P + PSNR(Irec, I)
    P = P / n
    return P

Ecrire la fonction qui à une ligne de la base de données précédente calcule le PSNR moyen sur 4 réalisations du bruit. Puis l'appliquer à la base de données et ajouter la colonne des PSNR calculés à la base de données.

In [ ]:
def row2DebruitTrans(row):
    I = imagesRef[row.Image]
    PSNR_moy = Debruit_Translat_PSNRMoyen(I, row.wave, row.Sigma, row.Seuil, row.NbT, 4)
    return PSNR_moy

In [ ]:
PSNR_moy = dfexp_DebruitTrans.apply(row2DebruitTrans, axis = 1)
print(PSNR_moy)

In [ ]:
dfexp_DebruitTrans['PSNR_moyen'] = PSNR_moy
print(dfexp_DebruitTrans)

Utiliser hvplot pour visualiser les résulatst contenus dans la base de données.

In [ ]:
h = HoverTool()
dfexp_DebruitTrans.hvplot('NbT','PSNR_moyen',by = 'wave', kind='scatter', groupby = ['Image','Seuil','Sigma']).opts(width=600, tools = [h]).redim.range(PSNR=(24.5,27),NbT=(0,6))

<span style = 'color:blue'>
    
Il semble que les ondelettes de Coifman 2 est en général plus efficace dans ce cas
    

# Quantification et Entropie de Shannon

In [ ]:
def ShannonEntropy(x):
    value,counts = np.unique(x, return_counts=True)
    Proba = counts / len(x)
    Ent = -np.sum(np.log2(Proba) * Proba)
    return Ent

In [ ]:
x = np.array([13,13,2,7,13,7,1,13])
print(ShannonEntropy(x))

In [ ]:
y = np.array([-2,-3,1,0,1,0,-2,-3])
print(ShannonEntropy(y))

Ecrire une fonction qui effectue la quantification de la transformée en ondelettes avec un pas "Pas". On pourra à nouveau utiliser la commande pywt.ravel_coeffs. La fonction doit renvoyer l'image calculée par quantification, le PSNR associé ainsi que le nombre d'octets estimé par la valeur de l'entropie a priori nécessaire pour coder une telle image. On considérera qu'on code séparément les coefficients d'échelle et les coefficients d'ondelettes. Tester la fonction.

In [ ]:
def QuantificationOndelettes(I,qmf,Pas):
    Lmax = pywt.dwt_max_level(len(I),qmf)
    L1 = Lmax
    W = pywt.wavedecn(I, qmf, mode='per', level=L1)
    n1, n2 = W[0].shape
    arr, coeff_slices = pywt.coeffs_to_array(W)
    coeff_echelle = arr[:n1*n2]
    if len(coeff_echelle) > 0:
        min_echelle,max_echelle = np.min(coeff_echelle),np.max(coeff_echelle)
        tab_echelle = np.arange(min_echelle,max_echelle,Pas)
        coeff_echelle = tab_echelle[np.digitize(coeff_echelle,tab_echelle)-1]
    else:
        coeff_echelle = []
        
    coeff_ondelette = arr[n1*n2:] 
    
    if len(coeff_ondelette) > 0:
        min_ondelette,max_ondelette = np.min(coeff_ondelette),np.max(coeff_ondelette)
        tab_ondelette = np.arange(min_ondelette,max_ondelette,Pas)
        coeff_ondelette = tab_ondelette[np.digitize(coeff_ondelette,tab_ondelette)-1]
    
    coeffs_from_arr = pywt.array_to_coeffs(np.concatenate((coeff_echelle,coeff_ondelette)), coeff_slices)
    Imagerec = pywt.waverecn(coeffs_from_arr,qmf,mode='per')
    Ent = ShannonEntropy(np.ravel(np.concatenate((coeff_echelle,coeff_ondelette))))
    
    return np.clip(Imagerec,0,255),Ent,PSNR(Imagerec,I)

In [ ]:
Irec,Ent,psnr = QuantificationOndelettes(imagesRef['Lenna'],'db4',10)

In [ ]:
strp1 = "%2.2f" % Ent
strp2 = "%2.2f" % psnr
te1 = 'Ent'
te2 = 'PSNR signal reconstruit'
TN = pn.Column(LaTeX(te1,size=15,dpi=100), LaTeX(strp1,size=15,dpi=100),LaTeX(te2,size=15,dpi=100),LaTeX(strp2,size=15,dpi=100))
        
pn.Row(TN,hv.Image(imagesRef['Lenna']).opts(cmap='gray',width=300,height=300)\
       ,hv.Image(Irec).opts(cmap='gray',width=300,height=300))

Créer le dashboard asscoié à la focntion précédente. 
Le dashboard doit renvoyer l'image quantifiée, le PSNR de l'image ainsi que le facteur de compression théorique associé. 

In [ ]:
class WaveQuant(param.Parameterized):
    wave = param.ObjectSelector(default="haar",objects=wavelist)
    QS = param.Number(30,bounds=(10,300))
    image = param.ObjectSelector(default="Canaletto",objects=imagesRef.keys())
    
    def view(self):
        Irec,Ent,psnr = QuantificationOndelettes(imagesRef[self.image],'db4',self.QS)
        strp1="%2.2f" % Ent
        strp2="%2.2f" % psnr
        te1='Ent'
        te2='PSNR signal reconstruit'
        TN = pn.Column(LaTeX(te1,size=15,dpi=100),LaTeX(strp1,size=15,dpi=100), LaTeX(te2,size=15,dpi=100),LaTeX(strp2,size=15,dpi=100))
        
        return pn.Row(TN,hv.Image(imagesRef[self.image]).opts(cmap='gray',width=300,height=300)
                       ,hv.Image(Irec).opts(cmap='gray',width=300,height=300))

In [ ]:
wavequant = WaveQuant()
pn.Column(wavequant.param,wavequant.view)

Créer dun plan d'expériences pour comparer les différentes ondelettes pour la quantification... et poursuivre jusqu'à obtenir un affichage de la base de données ainsi créée avec hvplot.

In [ ]:
experiences_quant = {'Image':imagesRef.keys(),'wave':wavelist,'QS':np.linspace(2,200,10)}
dfexperiences_quant = pd.DataFrame(list(itertools.product(*experiences_quant.values())),columns=experiences_quant.keys())
print(dfexperiences_quant)

In [ ]:
def row2DistorsionRate(row):
    Irec, Ent, psnr = QuantificationOndelettes(imagesRef[row.Image], row.wave, row.QS)
    p_real = PSNR(Irec,imagesRef[row.Image])
    return p_real

In [ ]:
result = dfexperiences_quant.apply(row2DistorsionRate, axis=1)
dfexperiences_quant['PSNR'] = result

In [ ]:
print(dfexperiences_quant)

In [ ]:
h = HoverTool()
dfexperiences_quant.hvplot('QS','PSNR',by='wave',kind='scatter',groupby=['Image']).opts(width=600,tools = [h]).redim.range(PSNR=(4,60),QS=(0,220))
